# Time Series - Tesla Stock Price ROI Analysis

Using Pandas time series to visualise Tesla stock price return of investment (ROI)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# read data
tsla = pd.read_csv('../input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')
tsla.head()

In [ ]:
# handle time series data by applying to_datetime function
tsla["Date"] = pd.to_datetime(tsla["Date"])

In [ ]:
tsla.info()

In [ ]:
# set `datetime` as index
tsla = tsla.set_index(keys="Date")
tsla.head()

In [ ]:
# plot graph for year 2015 to 2016
tsla['2015-01-01': '2016-12-31']['Close'].plot()

In [ ]:
# for simplicity, use just the closing price for remaining graph plotting exercise
tsla = tsla['Close']

In [ ]:
# resample data to at the end of business year:
tsla.plot(alpha=0.5, style='-')
tsla.resample('BA').mean().plot(style=':')
tsla.asfreq('BA').plot(style='--');
plt.legend(['input', 'resample', 'asfreq'],
           loc='upper left');

In [ ]:
# resample data to from business day frequency to daily frequency 
# note the two strategies for filling the gaps
fig, ax = plt.subplots(2, sharex=True)

# pick first ten days
data = tsla.iloc[:10]

data.asfreq('D').plot(ax=ax[0], marker='o')

# back-fill
data.asfreq('D', method='bfill').plot(ax=ax[1], style='-o')

# forward-fill
data.asfreq('D', method='ffill').plot(ax=ax[1], style='--o')

ax[1].legend(["back-fill", "forward-fill"]);

In [ ]:
# visualise shifting data by 900 days

fig, ax = plt.subplots(3, sharey=True)

# apply a frequency to the data
tsla = tsla.asfreq('D', method='pad')

tsla.plot(ax=ax[0])
tsla.shift(900).plot(ax=ax[1])
tsla.tshift(900).plot(ax=ax[2])

# legends and annotations
local_max = pd.to_datetime('2013-11-05')
offset = pd.Timedelta(900, 'D')

ax[0].legend(['input'], loc=2)
ax[0].get_xticklabels()[2].set(weight='heavy', color='red')
ax[0].axvline(local_max, alpha=0.3, color='red')

ax[1].legend(['shift(900)'], loc=2)
ax[1].get_xticklabels()[2].set(weight='heavy', color='red')
ax[1].axvline(local_max + offset, alpha=0.3, color='red')

ax[2].legend(['tshift(900)'], loc=2)
ax[2].get_xticklabels()[1].set(weight='heavy', color='red')
ax[2].axvline(local_max + offset, alpha=0.3, color='red');

In [ ]:
# plot return of investment ROI
ROI = 100 * (tsla.tshift(-365) / tsla - 1)
ROI.plot()
plt.ylabel('% Return on Investment');

In [ ]:
# rolling windows
rolling = tsla.rolling(365, center=True)
fifty_day_rolling = tsla.rolling(50, center=True)

data = pd.DataFrame({'input': tsla,
                     'one-year rolling_mean': rolling.mean(),
                     'one-year rolling_std': rolling.std(),
                     '50-day rolling_mean': fifty_day_rolling.mean()}
                   )
ax = data.plot(style=['-', '--', ':', '-,'], figsize=(10,10))
ax.lines[0].set_alpha(0.3)